In [1]:
import pandas as pd
import numpy as np
from pydataset import data
import seaborn as sns
# acquire
from env import user, password, hostname
from pydataset import data

# turn off pink warning boxes
import warnings
warnings.filterwarnings("ignore")

## 1. Attendance Data

Read the data from the attendance table and calculate an attendance percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

You should end up with something like this:




In [2]:
df_attendance = pd.read_csv('attendance.csv')

In [3]:
df_attendance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  4 non-null      object
 1   2018-01-01  4 non-null      object
 2   2018-01-02  4 non-null      object
 3   2018-01-03  4 non-null      object
 4   2018-01-04  4 non-null      object
 5   2018-01-05  4 non-null      object
 6   2018-01-06  4 non-null      object
 7   2018-01-07  4 non-null      object
 8   2018-01-08  4 non-null      object
dtypes: object(9)
memory usage: 416.0+ bytes


In [4]:
df_attendance.shape

(4, 9)

In [5]:
df_attendance.describe()

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
count,4,4,4,4,4,4,4,4,4
unique,4,2,2,3,4,3,2,3,2
top,Sally,P,T,T,H,P,T,P,T
freq,1,2,3,2,1,2,3,2,3


In [6]:
df_attendance.rename(columns={'Unnamed: 0': 'name'}, inplace=True)
df_attendance

,name,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [7]:
#In order to make each row a single observation,
#each row should represent a single date and single student
df_attendance = df_attendance.melt( id_vars='name',var_name='date', value_name='attend')
df_attendance

,name,date,attend
0,Sally,2018-01-01,P
1,Jane,2018-01-01,A
2,Billy,2018-01-01,A
3,John,2018-01-01,P
4,Sally,2018-01-02,T
5,Jane,2018-01-02,P
6,Billy,2018-01-02,T
7,John,2018-01-02,T
8,Sally,2018-01-03,T
9,Jane,2018-01-03,T


In [8]:
#Let's make the attendance value something we can work with
df_attendance['attend'] = df_attendance['attend'].map({'P': 1, 'H': 0.5, 'A': 0, 'T': 0.9})
df_attendance


,name,date,attend
0,Sally,2018-01-01,1.0
1,Jane,2018-01-01,0.0
2,Billy,2018-01-01,0.0
3,John,2018-01-01,1.0
4,Sally,2018-01-02,0.9
5,Jane,2018-01-02,1.0
6,Billy,2018-01-02,0.9
7,John,2018-01-02,0.9
8,Sally,2018-01-03,0.9
9,Jane,2018-01-03,0.9


In [9]:
#Now we can groupby name and get the mean
df_attendance.groupby('name').mean()

,attend
name,
Billy,0.5250
Jane,0.6875
John,0.9125
Sally,0.7625


In [10]:
attendance_data = [
    {"name": "Sally", "P": 2, "H": 1, "T": 4, 'A': 1},
    {"name": "Jane", "P": 1, "H": 0, "T": 5, 'A': 2},
    {"name": "Billy", "P": 1, "H": 1, "T": 3, 'A': 3},
    {"name": "John", "P": 5, "H": 1, "T": 2, 'A': 0}
]

def calculate_attendance_percentage(attendance_data):
    for record in attendance_data:
        full_days = record["P"]
        half_days = record["H"]
        tardies = record["T"]
        absences = record["A"] 
        

        # Calculate the attendance percentage
        total_days =  ((full_days * 1) + ( (8 - full_days) - (half_days * 0.5) -  (tardies * 0.1) - (absences)))
        attendance_percentage = (total_days) * 100/8

        # Add the attendance percentage to the record
        record["attendance_percentage"] = attendance_percentage

    return attendance_data

result = calculate_attendance_percentage(attendance_data)
for record in result:
    print(f"{record['name']}: {record['attendance_percentage']}%")


Sally: 76.25%
Jane: 68.75%
Billy: 52.5%
John: 91.25%


## 2. Coffee Levels

* Read the coffee_levels table.

In [11]:
df_coffee = pd.read_csv('coffee_levels.csv')

In [12]:
df_coffee.head()

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291


* Transform the data so that each carafe is in it's own column.

In [13]:
new_coffee = df_coffee.pivot(index='hour', columns='coffee_carafe', values= 'coffee_amount')
new_coffee

#new_coffee.columns = [f'{col[0]}_{col[1]}_{col[2]}' for col in new_coffee.columns]

coffee_carafe,x,y,z
hour,,,
8,0.816164,0.189297,0.999264
9,0.451018,0.521502,0.915990
10,0.843279,0.023163,0.144928
11,0.335533,0.235529,0.311495
12,0.898291,0.017009,0.771947
13,0.310711,0.997464,0.398520
14,0.507288,0.058361,0.864464
15,0.215043,0.144644,0.436364
16,0.183891,0.544676,0.280621


* Is this the best shape for the data?

No, the data is tidy, but unclear information. 

## 3. Cake Recipes

a. Read the cake_recipes table. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.

In [14]:
df_cake = pd.read_csv('cake_recipes.csv')
df_cake

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084
5,c:top,71.306308,82.795477,92.098049,53.960273
6,d:bottom,52.799753,58.670419,51.747686,56.183110
7,d:top,96.873178,76.101363,59.571620,50.971626


In [15]:
df_cake.shape

(8, 5)

In [16]:
df_cake.dtypes

recipe:position     object
225                float64
250                float64
275                float64
300                float64
dtype: object

b. Tidy the data as necessary.

In [17]:
df_cake.rename(columns={'recipe:position': 'recipe'}, inplace=True)
df_cake

,recipe,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084
5,c:top,71.306308,82.795477,92.098049,53.960273
6,d:bottom,52.799753,58.670419,51.747686,56.183110
7,d:top,96.873178,76.101363,59.571620,50.971626


In [18]:
#Let's start with making a single column of temp 
#that contains temp values
new_cake = df_cake.melt(id_vars='recipe', var_name='temp', value_name='score')
new_cake


,recipe,temp,score
0,a:bottom,225,61.738655
1,a:top,225,51.709751
2,b:bottom,225,57.095320
3,b:top,225,82.455004
4,c:bottom,225,96.470207
5,c:top,225,71.306308
6,d:bottom,225,52.799753
7,d:top,225,96.873178
8,a:bottom,250,53.912627
9,a:top,250,52.009735


In [19]:
tidy_cake = new_cake.recipe.str.split(pat=':', expand=True)
tidy_cake.columns = ['Recipe', 'Position']
tidy_cake

,Recipe,Position
0,a,bottom
1,a,top
2,b,bottom
3,b,top
4,c,bottom
5,c,top
6,d,bottom
7,d,top
8,a,bottom
9,a,top


In [20]:
tidy_cake2 = pd.concat([new_cake, tidy_cake], axis=1)
tidy_cake2

,recipe,temp,score,Recipe,Position
0,a:bottom,225,61.738655,a,bottom
1,a:top,225,51.709751,a,top
2,b:bottom,225,57.095320,b,bottom
3,b:top,225,82.455004,b,top
4,c:bottom,225,96.470207,c,bottom
5,c:top,225,71.306308,c,top
6,d:bottom,225,52.799753,d,bottom
7,d:top,225,96.873178,d,top
8,a:bottom,250,53.912627,a,bottom
9,a:top,250,52.009735,a,top


In [22]:
tidy_cake2 = tidy_cake2.drop(columns='recipe')
tidy_cake2

,temp,score,Recipe,Position
0,225,61.738655,a,bottom
1,225,51.709751,a,top
2,225,57.095320,b,bottom
3,225,82.455004,b,top
4,225,96.470207,c,bottom
5,225,71.306308,c,top
6,225,52.799753,d,bottom
7,225,96.873178,d,top
8,250,53.912627,a,bottom
9,250,52.009735,a,top


c. Which recipe, on average, is the best?

In [28]:
#Getting the recipe with the highest average tastiness rating
tidy_cake2.groupby('Recipe').mean().idxmax()

score    b
dtype: object

d. Which oven temperature, on average, produces the best results?

In [40]:
best_recipe = tidy_cake2.groupby('Recipe').mean().max()
best_recipe

score    76.736074
dtype: float64

e. Which combination of recipe, rack position, and temperature gives the best result?

In [38]:
best_cake = tidy_cake2.groupby(['Recipe','Position','temp']).mean().idxmax()
best_cake

score    (b, bottom, 300)
dtype: object